In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import bp
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
U = bp.variableNode("Unblocked FEV1", 2, 6, 0.1)
B = bp.variableNode("Small airway blockage", 0, 1, 0.1)
FEV1 = bp.variableNode("FEV1", 0, 6, 0.1)

In [3]:
graph = BayesianNetwork([(U.name, FEV1.name), (B.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins)-1,
    values=bp.calc_pgmpy_cpt(U, B, FEV1).tolist(),
    evidence=[U.name, B.name],
    evidence_card=[len(U.bins)-1, len(B.bins)-1],
)

prior_b = TabularCPD(
    variable=B.name,
    variable_card=len(B.bins)-1,
    values=B.marginal(B),
    evidence=[],
    evidence_card=[],
)

prior_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins)-1,
    values=U.marginal(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, prior_b, prior_u)

graph.check_model()


calculating cpt of shape 60 x 40 x 10 (C x A x B) 


True

In [5]:
bp = BeliefPropagation(graph)
bp.query(variables=[U.name, B.name], evidence={FEV1.name: 3})

0it [00:00, ?it/s]

<DiscreteFactor representing phi(Small airway blockage:5, Unblocked FEV1:4) at 0x7fd262335360>